<a href="https://colab.research.google.com/github/adenning-1/QNC-2025/blob/main/Parametric_Correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Plot X vs Y. Do they look related?
2. Calculate
 and
, first using the equations above and then using either the Python numpy funciton corrcoef or Matlab's built-in corrcoef. Did you get the same answers?
3. What is the standard error of
? The 95% confidence intervals computed from the standard error?
4. Should the value of
 be considered significant at the p<0.05 level, given a two-tailed test (i.e., we reject if the test statistic is too large on either tail of the null distribution) for
?
5. Yale does the exact same study and finds that his correlation value is 0.75. Is this the same as yours? That is, evaluate
.
6. Finally, calculate the statistical power and sample size needed to reject



In [8]:
# Setup measured variables
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Setup DF
data = {"wing" : [10.4, 10.8, 11.1, 10.2, 10.3, 10.2, 10.7, 10.5, 10.8, 11.2, 10.6, 11.4],
        "tail" : [7.4, 7.6, 7.9, 7.2, 7.4, 7.1, 7.4, 7.2, 7.8, 7.7, 7.8, 8.3]}
data = pd.DataFrame(data)

# Plot
plt(data["wing"], data["tail"])

TypeError: 'module' object is not callable